In [344]:
import pandas as pd
import numpy as np
import seaborn as sns

In [345]:
df = pd.read_csv('tidy_2.csv',encoding='utf-8')

## 새로운 칼럼 만들기

### 1) 대상인원 칼럼 만들기

In [346]:
df['대상인원'] = df['대상인원'].copy().str.extract(r'[\s|외|총|등](\d+)\s?[명|인]?')
# 대상인원이 없는 경우 운전기사 회식, 격려금 지급 등 맛집과 상관없는 경우가 많으니 삭제
df = df.dropna(axis=0, subset=['대상인원'], how='all')
df['대상인원'] = df['대상인원'].copy().astype(int)


### 2) 년도별로 그룹화하기

In [347]:
df_2016 = df[df['year'] == 2016]
df_2017 = df[df['year'] == 2017]
df_2018 = df[df['year'] == 2018]
df_2019 = df[df['year'] == 2019]
df_2020 = df[df['year'] == 2020]
df_2021 = df[df['year'] == 2021]

dfs = [df, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021]
df_2016.shape, df_2017.shape, df_2018.shape, df_2019.shape, df_2020.shape, df_2021.shape

((498, 15), (1500, 15), (1370, 15), (2067, 15), (1560, 15), (721, 15))

### 3) 집행지명으로 그룹화하기

In [348]:
def group_place(df):
    df_group = df.groupby('집행지명')\
            ['집행지명', 'cnt', '집행금액', '대상인원']\
            .sum().reset_index()
    return df_group

df_2016_group = group_place(df_2016)
df_2017_group = group_place(df_2017)
df_2018_group = group_place(df_2018)
df_2019_group = group_place(df_2019)
df_2020_group = group_place(df_2020)
df_2021_group = group_place(df_2021)
df_group = group_place(df)

df_groups = [df_group, df_2016_group, df_2017_group, df_2018_group, df_2019_group, df_2020_group, df_2021_group]
df_group.shape, df_2016_group.shape, df_2017_group.shape, df_2018_group.shape, df_2019_group.shape, df_2020_group.shape, df_2021_group.shape

<ipython-input-348-3ed1c11bd71d>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_group = df.groupby('집행지명')\


((1679, 4), (272, 4), (473, 4), (500, 4), (556, 4), (520, 4), (299, 4))

In [349]:
# 회당, 인당 금액을 구해보자. 
def price_per_num_people(df_group):
    df_group['회당집행금액'] = df_group['집행금액'] / df_group['cnt']
    df_group['인당집행금액'] = df_group['집행금액'] / df_group['대상인원']
    df_group['회당명수'] = df_group['대상인원'] / df_group['cnt']

for df_group in df_groups:
    df_group = price_per_num_people(df_group)

df_groups[0].head()

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
0,(주)강가,15,302700,13,20180.000000,23284.615385,0.866667
1,(주)골방집,3,20000,2,6666.666667,10000.000000,0.666667
2,(주)광화문아띠,5,116000,4,23200.000000,29000.000000,0.800000
3,(주)광화문아띠(친니&바심),24,480000,23,20000.000000,20869.565217,0.958333
4,(주)나무와벽돌,8,97200,6,12150.000000,16200.000000,0.750000


# 서울시 진짜 맛집 프로젝트

## 카테고리별 맛집

### 1. 가장 자주 간 식당 (전체 공무원)

In [350]:
def by_cnt(df_group):
    df_group = df_group.sort_values(by='cnt', ascending=False)
    df_group['회당명수'] = round(df_group['회당명수']).astype(int)
    df_group['회당집행금액'] = round(df_group['회당집행금액']).astype(int)
    df_group['인당집행금액'] = round(df_group['인당집행금액']).astype(int)
    df_group['집행금액'] = round(df_group['집행금액']).astype(int)
    return df_group

for i, df_group in enumerate(df_groups):
    df_groups[i] = by_cnt(df_group)

In [360]:
# 전체 공무원
df_groups[0].head(3)
totla=df_groups[0].head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1334,참숯골,1137,19053100,982,16757,19402,1
523,만복림,766,16694400,670,21794,24917,1
608,바닷가작은부엌,700,15119400,604,21599,25032,1


In [352]:
# 2016년
df_groups[1].head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
58,던킨도너츠,501,2000000,500,3992,4000,1
88,몽중헌,168,5224000,150,31095,34827,1
51,달개비자연음식,143,4091700,128,28613,31966,1


In [353]:
# 2017년
df_groups[2].head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
373,참숯골,237,3537700,204,14927,17342,1
188,빚은,236,1621000,229,6869,7079,1
275,열빈,229,4696000,210,20507,22362,1


In [354]:
# 2018년
df_groups[3].head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
302,연남빠니니,500,1940000,499,3880,3888,1
147,만복림,303,7110800,274,23468,25952,1
181,바닷가작은부엌,290,6679000,248,23031,26931,1


In [355]:
# 2019년
df_groups[4].head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
339,엔젤호두파이,601,2640000,600,4393,4400,1
457,참숯골,414,6852000,358,16551,19140,1
190,만복림,394,8233100,335,20896,24576,1


In [356]:
# 2020년
df_groups[5].head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
318,엄마손마트,300,2485600,299,8285,8313,1
22,㈜바닷가작은부엌덕수궁점,297,5597000,259,18845,21610,1
197,배재반점,284,6331000,246,22292,25736,1


In [278]:
# 2021년도
df_groups[6].head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
6,㈜바닷가작은부엌덕수궁점,101,2590100,79,25645,32786,1
50,대복,95,1933000,73,20347,26479,1
152,송탄영빈루,89,1651000,66,18551,25015,1


### 2. 가장 돈을 많이 쓴 식당 

In [279]:
# 전체
df_groups[0].sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1334,참숯골,1137,19053100,982,16757,19402,1
523,만복림,766,16694400,670,21794,24917,1
608,바닷가작은부엌,700,15119400,604,21599,25032,1


In [280]:
# 2016년
df_groups[1].sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
88,몽중헌,168,5224000,150,31095,34827,1
51,달개비자연음식,143,4091700,128,28613,31966,1
139,식객촌,116,3592000,103,30966,34874,1


In [281]:
# 2017년
df_groups[2].sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
275,열빈,229,4696000,210,20507,22362,1
373,참숯골,237,3537700,204,14927,17342,1
107,동해수산,163,3256000,139,19975,23424,1


In [282]:
# 2018년
df_groups[3].sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
147,만복림,303,7110800,274,23468,25952,1
181,바닷가작은부엌,290,6679000,248,23031,26931,1
122,동해일식,170,4075000,139,23971,29317,1


In [283]:
# 2019년
df_groups[4].sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
190,만복림,394,8233100,335,20896,24576,1
153,동원참치,373,8044000,320,21566,25138,1
468,초류향,363,7858500,312,21649,25188,1


In [284]:
# 2020년
df_groups[5].sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
197,배재반점,284,6331000,246,22292,25736,1
22,㈜바닷가작은부엌덕수궁점,297,5597000,259,18845,21610,1
117,동원VIP참치,217,5253000,185,24207,28395,1


In [285]:
# 2021년도
df_groups[6].sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
6,㈜바닷가작은부엌덕수궁점,101,2590100,79,25645,32786,1
50,대복,95,1933000,73,20347,26479,1
62,동원VIP참치서소문점,85,1800000,63,21176,28571,1


### 3. 회당 집행금액이 가장 많은 식당
(30회 이상 방문, 전체 공무원)

In [286]:
for i, df_group in enumerate(df_groups):
    df_groups[i] = df_group[df_group['cnt']>30]

In [287]:
# 전체
df_groups[0].sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
234,김가네,32,1823000,26,56969,70115,1
313,능금산장,39,1875000,74,48077,25338,2
1456,통인가게,153,6800000,135,44444,50370,1


In [288]:
# 2016년
df_groups[1].sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
226,통인가게,43,1950000,39,45349,50000,1
202,지화자,32,1132000,29,35375,39034,1
228,파로,51,1777200,46,34847,38635,1


In [289]:
# 2017년 
df_groups[2].sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
412,통인가게,60,2650000,53,44167,50000,1
11,YORI,43,1588480,42,36941,37821,1
37,광화문양갈비,39,1239400,44,31779,28168,1


In [290]:
# 2018년
df_groups[3].sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
366,장호왕곱창,60,2222500,51,37042,43578,1
195,베르가모,99,2952000,98,29818,30122,1
497,ＡＷ 컨벤션센터,55,1635000,54,29727,30278,1


In [291]:
# 2019년
df_groups[4].sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
493,통인가게,32,1400000,28,43750,50000,1
452,차원,32,950000,30,29688,31667,1
530,향연,31,877900,28,28319,31354,1


In [292]:
# 2020년
df_groups[5].sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
10,(주)태맥을지점,59,1666900,57,28253,29244,1
83,남원식당,75,2067000,69,27560,29957,1
426,차이797,86,2346500,78,27285,30083,1


In [293]:
# 2021년
df_groups[6].sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
250,초류향,31,842000,23,27161,36609,1
120,북창갈비,32,858000,24,26812,35750,1
6,㈜바닷가작은부엌덕수궁점,101,2590100,79,25645,32786,1


### 4. 인당 집행금액이 가장 많은 식당

In [294]:
# 전체
df_groups[0].sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
234,김가네,32,1823000,26,56969,70115,1
1456,통인가게,153,6800000,135,44444,50370,1
1296,지화자,32,1132000,29,35375,39034,1


In [295]:
# 2016
# 김영란법 생기기 전인가? 3만원이 넘는다. 
df_groups[1].sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
226,통인가게,43,1950000,39,45349,50000,1
202,지화자,32,1132000,29,35375,39034,1
228,파로,51,1777200,46,34847,38635,1


In [296]:
# 2017
df_groups[2].sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
412,통인가게,60,2650000,53,44167,50000,1
11,YORI,43,1588480,42,36941,37821,1
7,HITEMOSKVA,48,1504980,47,31354,32021,1


In [297]:
# 2018
df_groups[3].sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
366,장호왕곱창,60,2222500,51,37042,43578,1
483,（주）달개비자연음식,116,3013000,99,25974,30434,1
64,남산집,49,1185000,39,24184,30385,1


In [298]:
# 2019
df_groups[4].sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
493,통인가게,32,1400000,28,43750,50000,1
452,차원,32,950000,30,29688,31667,1
530,향연,31,877900,28,28319,31354,1


In [299]:
# 2020
df_groups[5].sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
341,오륙도,43,1104000,36,25674,30667,1
440,철철복집,40,972000,32,24300,30375,1
426,차이797,86,2346500,78,27285,30083,1


In [300]:
# 2021
df_groups[6].sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
250,초류향,31,842000,23,27161,36609,1
120,북창갈비,32,858000,24,26812,35750,1
6,㈜바닷가작은부엌덕수궁점,101,2590100,79,25645,32786,1


### 5. 가장 많은 사람이 간 식당

In [301]:
# 전체
df_groups[0].sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1334,참숯골,1137,19053100,982,16757,19402,1
523,만복림,766,16694400,670,21794,24917,1
608,바닷가작은부엌,700,15119400,604,21599,25032,1


In [302]:
# 2016
df_groups[1].sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
58,던킨도너츠,501,2000000,500,3992,4000,1
88,몽중헌,168,5224000,150,31095,34827,1
221,코코호도,143,916000,140,6406,6543,1


In [303]:
# 2017
df_groups[2].sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
188,빚은,236,1621000,229,6869,7079,1
275,열빈,229,4696000,210,20507,22362,1
373,참숯골,237,3537700,204,14927,17342,1


In [304]:
# 2018
df_groups[3].sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
302,연남빠니니,500,1940000,499,3880,3888,1
147,만복림,303,7110800,274,23468,25952,1
181,바닷가작은부엌,290,6679000,248,23031,26931,1


In [305]:
# 2019
df_groups[4].sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
339,엔젤호두파이,601,2640000,600,4393,4400,1
457,참숯골,414,6852000,358,16551,19140,1
190,만복림,394,8233100,335,20896,24576,1


In [306]:
# 2020
df_groups[5].sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
318,엄마손마트,300,2485600,299,8285,8313,1
22,㈜바닷가작은부엌덕수궁점,297,5597000,259,18845,21610,1
197,배재반점,284,6331000,246,22292,25736,1


In [307]:
# 2021
df_groups[6].sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
6,㈜바닷가작은부엌덕수궁점,101,2590100,79,25645,32786,1
50,대복,95,1933000,73,20347,26479,1
152,송탄영빈루,89,1651000,66,18551,25015,1


### 6. 단체손님이 많이 가는 식당
8명 이상의 단체손님이 가는 식당 중 가장 자주 가는 식당

In [308]:
# 전체
df_groups[0][df_groups[0]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수


In [309]:
# 2016년
# 2016년엔 단체로 가지 않은 것일까, 데이터가 적은 것일까
df_groups[1][df_groups[1]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수


In [310]:
# 2017년
df_groups[2][df_groups[2]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수


In [311]:
# 2018년
# 2016, 2017년에 사랑받던 제일가든이 없어지고 열빈이 대세가됨
df_groups[3][df_groups[3]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수


In [312]:
df_groups[4][df_groups[4]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수


In [313]:
df_groups[5][df_groups[5]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수


In [314]:
df_groups[6][df_groups[6]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수


## 부서별로 살펴보기

In [315]:
# 부서별로 묶기
def make_divs(df):
    df_divs = df.groupby('소속부서1')['cnt', 'num', '집행금액']\
            .sum().reset_index()
    # 인당/회당 금액 내기
    df_divs['인당집행금액'] = df_divs['집행금액'] / df_divs['num']
    df_divs['회당집행금액'] = df_divs['집행금액'] / df_divs['cnt']
    df_divs['인당집행금액'] = round(df_divs['인당집행금액']).astype(int)
    df_divs['회당집행금액'] = round(df_divs['회당집행금액']).astype(int)
    return df_divs

df_divs=[]
for i, df in enumerate(dfs):
    df_divs.append(make_divs(df))

<ipython-input-315-fffd74038078>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_divs = df.groupby('소속부서1')['cnt', 'num', '집행금액']\


### 1. 가장 회식을 자주 한 부서

In [316]:
# 전체
df_divs[0].sort_values(by='cnt', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,55441,7707,1011714832,131272,18248


In [317]:
# 2016
df_divs[1].sort_values(by='cnt', ascending=False).head(5)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,4987,477,104135000,218312,20881


In [318]:
# 2017
df_divs[2].sort_values(by='cnt', ascending=False).head(5)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,11802,1500,207158860,138106,17553


In [319]:

df_divs[3].sort_values(by='cnt', ascending=False).head(5)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,10329,1370,206881063,151008,20029


In [320]:
df_divs[4].sort_values(by='cnt', ascending=False).head(5)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,15642,2067,273185421,132165,17465


In [321]:
df_divs[5].sort_values(by='cnt', ascending=False).head(5)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,9903,1560,168626858,108094,17028


In [322]:
df_divs[6].sort_values(by='cnt', ascending=False).head(5)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,2668,721,49529830,68696,18564


### 2. 가장 돈을 많이 쓴 부서

In [323]:
# 전체
df_divs[0].sort_values(by='집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,55441,7707,1011714832,131272,18248


In [324]:
# 2016
df_divs[1].sort_values(by='집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,4987,477,104135000,218312,20881


In [325]:
# 2017
df_divs[2].sort_values(by='집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,11802,1500,207158860,138106,17553


In [326]:
# 2018
df_divs[3].sort_values(by='집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,10329,1370,206881063,151008,20029


In [327]:

df_divs[4].sort_values(by='집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,15642,2067,273185421,132165,17465


In [328]:

df_divs[5].sort_values(by='집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,9903,1560,168626858,108094,17028


In [329]:

df_divs[6].sort_values(by='집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,2668,721,49529830,68696,18564


### 3. 가장 비싼 것을 먹는 부서

In [330]:
# 전체
df_divs[0].sort_values(by='인당집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,55441,7707,1011714832,131272,18248


In [331]:
# 2016년
# 역시 김영란법 시행 전이라 3만원이 넘는다. 
df_divs[1].sort_values(by='인당집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,4987,477,104135000,218312,20881


In [332]:
# 2017년
# 라고 생각했는데 시행 후에도 3만원 넘는것 무엇...
df_divs[2].sort_values(by='인당집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,11802,1500,207158860,138106,17553


In [333]:
# 2018년
# 인당 7만 5천원?? 나중에 부시장 데이터 꼼꼼히 살펴보기
df_divs[3].sort_values(by='인당집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,10329,1370,206881063,151008,20029


In [334]:
df_divs[4].sort_values(by='인당집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,15642,2067,273185421,132165,17465


In [335]:
df_divs[5].sort_values(by='인당집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,9903,1560,168626858,108094,17028


In [336]:
df_divs[6].sort_values(by='인당집행금액', ascending=False).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,2668,721,49529830,68696,18564


### 4. 가장 싼 것을 먹는 부서

In [337]:
# 전체
df_divs[0].sort_values(by='인당집행금액', ascending=True).head(3)
# 인당 6000원대라니 8배 차이나는 이 빈부격차 무엇..

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,55441,7707,1011714832,131272,18248


In [338]:
# 2016년
df_divs[1].sort_values(by='인당집행금액', ascending=True).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,4987,477,104135000,218312,20881


In [339]:
# 2017년
df_divs[2].sort_values(by='인당집행금액', ascending=True).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,11802,1500,207158860,138106,17553


In [340]:
# 2018년
# 재무국이 꾸준하게 하위권. 시의 예산을 보면 차마 비싼 것을 먹을 수 없는 것일까
df_divs[3].sort_values(by='인당집행금액', ascending=True).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,10329,1370,206881063,151008,20029


In [341]:
df_divs[4].sort_values(by='인당집행금액', ascending=True).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,15642,2067,273185421,132165,17465


In [342]:
df_divs[5].sort_values(by='인당집행금액', ascending=True).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,9903,1560,168626858,108094,17028


In [343]:
df_divs[6].sort_values(by='인당집행금액', ascending=True).head(3)

,소속부서1,cnt,num,집행금액,인당집행금액,회당집행금액
0,감사위원회,2668,721,49529830,68696,18564
